<a href="https://colab.research.google.com/github/DSBA-6190-Final-Project-Team/DSBA-6190_Final-Project/blob/master/State_Farm_Distracted_Driver_Process_Flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Background and Proposed Method

## Data
The data set is the [State Farm Distracted Driver](https://www.kaggle.com/c/state-farm-distracted-driver-detection) dataset hosted on Kaggle.

The Kaggle page provides three items: 
1. a list of training images, their subject (driver) id, and class id (CSV)
2. a sample_submission.csv - a sample submission file in the correct format (CSV)
3. zipped folder of all (train/test) images (ZIP)

The data that the algorithm will train on is in the zipped folder. Within the folder are two sub-directories, train and test. The train directory contains the training set of the data, with each class of image within its own subfolder. The test directory contains only images, not seperated by class. It is to be used to score the final algorithm for Kaggle.






## Environment
The current plan is to generate a model in Amazon Sagemaker. A Amazon Sagemaker Notebook, using the Sagemaker Image Classification algorithm, will be used to generate a model. The Notebook will also deplot the model, once trained.

## Algorithm
The Amazon Sagemaker Image Classification Algorithm has four main inputs:
1. training set of images
2. test set of images
3. lst/rec file for trianing set
4. lst/rec file for test set

The lst or rec file (either acceptable) functions as a mapping resource, connecting each image to the image location and image class.

## Data Preparation
The provided test set will not be useful as a test set for the algorithm. The Sagemaker Image Classification Algorithm requires a labeled test set. Therefore, we will need to derive our own test set from the provided training set.

Based on the [Towards Data blog post](https://towardsdatascience.com/how-i-tackled-my-first-kaggle-challenge-using-deep-learning-part-1-b0da29e1351b) walking through the same data, we are not going to split the training/test data strictly randomly. Instead, we will split the drivers randomly, so that a unique driver only exists in either the training or the test set.

### Training / Test Set
The first step will be developing a list of unique drivers. We should also check the amount of pictures per driver to get an idea of how even the split of pictures per driver is. 

Then, we will generate a list of training and test drivers by randomly selecting from the unique list of drivers, using a 80/20 split.

### LST/REC File Development
The Image Classification Algorithm requires a file which maps the  images to the associated classes, for the training and testing sets. This can be in the form of a lst or rec file. 

There is python script available [here](https://mxnet.apache.org/api/faq/recordio) which will create these files. But these files must on your machine locally. I have already exported the files to S3. Therefore, I will need to redownload the files to a local machine and run this script. 

#### Training / Test in the Mapping File
Ideally, the training and test files would be physically seperated into two seperate folders. I would prefer not to do that, as the image folders have already been uploaded to S3. 

Therefore, I am going to try to keep the training and tes images in the same folder, but develop two mapping documents (as is required). That way, the mapping documents will scan the same image folder, but identify only the images on the training or testing mapping document.

If this doesn't work I will revert to physiclaly seperating the training and test images.



# Import

## Library / Packages

In [0]:
import pandas as pd
import random 

## Data

In [0]:
url = "https://raw.githubusercontent.com/DSBA-6190-Final-Project-Team/DSBA-6190_Final-Project/master/wine_predict/data/driver_imgs_list.csv"

df_driver_index = pd.read_csv( url) 

# EDA

In [114]:
df_driver_index.shape

(22424, 3)

In [115]:
df_driver_index.head(5)

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [116]:
df_driver_index.columns

Index(['subject', 'classname', 'img'], dtype='object')

The following lists each unique driver along with the number of different classname and images are associated with each. Note the number of classnames is not unique, so the images an classnames have equal frequencies.

In [117]:
drivers_gb = df_driver_index.groupby(['subject'])
drivers_gb.count()

,classname,img
subject,,
p002,725,725
p012,823,823
p014,876,876
p015,875,875
p016,1078,1078
p021,1237,1237
p022,1233,1233
p024,1226,1226
p026,1196,1196


We will set the training/test split ratio.

In [0]:
train_test_split = 0.3

To split the data into training and test sets, we'll need a unique index of drivers. We don't need the frequency counts.

We will create a shuffled list of unique drivers.


In [138]:
drivers_unique = drivers_gb.groups.keys()
drivers_unique = list(drivers_unique)

random.Random(5590).shuffle(drivers_unique)
print(drivers_unique)

['p049', 'p022', 'p012', 'p075', 'p045', 'p072', 'p026', 'p042', 'p015', 'p039', 'p047', 'p081', 'p052', 'p014', 'p021', 'p035', 'p061', 'p064', 'p066', 'p056', 'p002', 'p041', 'p024', 'p016', 'p051', 'p050']


Now we'll set the list of drivers in the train set and the test set.

In [0]:
num_drivers_test = round(len(drivers_unique)*train_test_split)
#print(num_drivers_test)
num_drivers_train = len(drivers_unique) - num_drivers_test
#print(num_drivers_train)

In [136]:
drivers_test = drivers_unique[:num_drivers_test]
print(drivers_test)

['p049', 'p022', 'p012', 'p075', 'p045', 'p072', 'p026', 'p042']


In [137]:
drivers_train = drivers_unique[-num_drivers_train:]
print(drivers_train)

['p015', 'p039', 'p047', 'p081', 'p052', 'p014', 'p021', 'p035', 'p061', 'p064', 'p066', 'p056', 'p002', 'p041', 'p024', 'p016', 'p051', 'p050']


# Training and Test Image Lists
We'll now create two lists, one list of every image file name associated with the trainging set, another list associated with the test set. 

We will use these lists to filter the overall lst mapping file.

In [0]:
df_images_test = df_driver_index[df_driver_index['subject'].isin(drivers_test)]
df_images_training = df_driver_index[~df_driver_index['subject'].isin(drivers_test)]

In [144]:
df_images_training

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg
...,...,...,...
22419,p081,c9,img_56936.jpg
22420,p081,c9,img_46218.jpg
22421,p081,c9,img_25946.jpg
22422,p081,c9,img_67850.jpg


In [145]:
df_images_test

,subject,classname,img
725,p012,c0,img_10206.jpg
726,p012,c0,img_27079.jpg
727,p012,c0,img_50749.jpg
728,p012,c0,img_97089.jpg
729,p012,c0,img_37741.jpg
...,...,...,...
21596,p075,c9,img_15827.jpg
21597,p075,c9,img_16688.jpg
21598,p075,c9,img_64532.jpg
21599,p075,c9,img_7918.jpg
